In [ ]:
import pandas as pd
import glob
import os
import warnings

warnings.filterwarnings("ignore")

# =========================================================
# 1. 파일 자동 탐색 (이름이 조금 달라도 찾아냄)
# =========================================================
# 기본 폴더 경로
base_dir = os.path.join(os.getcwd(), "Gwanak_Data")

# 찾고 싶은 파일의 '핵심 단어'
target_keyword = "관악구_맛집_최종_10회이상"

print(f"🕵️‍♂️ 폴더에서 '{target_keyword}' 파일 찾는 중...")

# 폴더 내의 모든 파일을 뒤져서, 저 단어가 포함된 파일을 찾습니다.
candidates = glob.glob(os.path.join(base_dir, f"*{target_keyword}*"))

if not candidates:
    print("❌ 파일을 못 찾겠습니다! 폴더 경로가 맞는지 확인해주세요.")
    print(f"경로: {base_dir}")
    exit()

# 가장 유력한 파일 하나 선택
real_file_path = candidates[0]
print(f"👉 찾았다! 실제 파일 이름: {os.path.basename(real_file_path)}")

# =========================================================
# 2. 파일 읽기 (CSV? XLSX? 자동 판단)
# =========================================================
try:
    if real_file_path.endswith('.csv') or 'Sheet1' in real_file_path:
        print("📄 형식이 CSV네요. CSV로 읽습니다.")
        try:
            df = pd.read_csv(real_file_path, encoding='utf-8')
        except:
            df = pd.read_csv(real_file_path, encoding='cp949') # 한글 깨짐 방지
    else:
        print("📊 형식이 Excel이네요. 엑셀로 읽습니다.")
        df = pd.read_excel(real_file_path)

    print(f"✅ 데이터 로딩 성공! (총 {len(df)}개 식당)")

except Exception as e:
    print(f"❌ 읽기 실패. 원인: {e}")
    exit()

# =========================================================
# 3. 시트 나누기 및 저장
# =========================================================
save_path = os.path.join(base_dir, "관악구_맛집_심화분석_완료.xlsx")
print(f"💾 '{save_path}' 생성 중...")

try:
    with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
        
        # [시트 1] 전체 랭킹
        if '총방문횟수' in df.columns:
            rank_total = df.sort_values(by='총방문횟수', ascending=False)
            rank_total.to_excel(writer, sheet_name='1_전체랭킹', index=False)
        
        # [시트 2] 최근 1년 핫플
        if '최근1년_방문수' in df.columns:
            rank_recent = df[df['최근1년_방문수'] > 0].sort_values(by='최근1년_방문수', ascending=False)
            rank_recent.to_excel(writer, sheet_name='2_최근1년_핫플', index=False)

        # [시트 3] 점심 랭킹
        if '점심_방문수' in df.columns:
            rank_lunch = df[df['점심_방문수'] > 0].sort_values(by='점심_방문수', ascending=False)
            rank_lunch.to_excel(writer, sheet_name='3_점심_랭킹', index=False)

        # [시트 4] 저녁 랭킹
        if '저녁_방문수' in df.columns:
            rank_dinner = df[df['저녁_방문수'] > 0].sort_values(by='저녁_방문수', ascending=False)
            rank_dinner.to_excel(writer, sheet_name='4_저녁_랭킹', index=False)

        # [시트 5] 가성비
        if '가성비_방문수' in df.columns:
            rank_cheap = df[df['가성비_방문수'] > 0].sort_values(by='가성비_방문수', ascending=False)
            rank_cheap.to_excel(writer, sheet_name='5_가성비_1만원대', index=False)

        # [시트 6] 고급
        if '고급_방문수' in df.columns:
            rank_expensive = df[df['고급_방문수'] > 0].sort_values(by='고급_방문수', ascending=False)
            rank_expensive.to_excel(writer, sheet_name='6_고급_3만원이상', index=False)

    print(f"\n🎉 [성공] 완벽하게 저장되었습니다!")
    print(f"📂 파일 위치: {save_path}")

except PermissionError:
    print("\n🚨 [에러] 엑셀 파일이 켜져 있습니다! 끄고 다시 해주세요.")